In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#Read csv file
df = pd.read_csv("./books.csv")

In [3]:
#view first 5 raws
df.head(5)

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0


In [4]:
#columns of csv
df.columns

Index(['isbn13', 'isbn10', 'title', 'subtitle', 'authors', 'categories',
       'thumbnail', 'description', 'published_year', 'average_rating',
       'num_pages', 'ratings_count'],
      dtype='object')

In [5]:
df.shape

(6810, 12)

In [6]:
df['description'][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [7]:
#removing short word
def short_word(columns,word):
    list_word = []
    for i in range(len(columns)):
        phrase = str(columns[i]).lower()
        phrase = phrase.replace(",","")
        phrase = phrase.replace(".","")
        phrase = phrase.split(" ")
        phrase = np.array(phrase)
        phrase = np.unique(phrase)
        phrase = list(phrase)
        for i in word:
            if i in phrase:
                phrase.remove(i)
        phrase = (" ".join(phrase)).replace("'","")
        list_word.append(phrase)
    return list_word

In [8]:
#stop words
word = ['to','all','into','the','once','no','is','and','due','for','in','of','with','be','With','more','may','a','as','a','from','any','up','has','have','use'
        ,'been','will','she','he','they','it','them','who','between','his','her','that','was','were','for','over','at','an']

In [9]:
#filter the text
text = short_word(df['description'],word)
#add filtered text data into csv
df['filtered_description'] = text
print(df['filtered_description'][2])

acclaimed anti-hero compelling covenant donaldsons featuing second stephen thomas trilogy two volume


In [10]:
print(df.shape)
df.dropna()
print(df.shape)

(6810, 13)
(6810, 13)


In [11]:
#vectorize the data
cv = CountVectorizer()
count = cv.fit_transform(df['filtered_description'])

In [12]:
#determine the smilarity
similarity = cosine_similarity(count)

In [13]:
#find the index of title
def get_index(title):
  return df[df.title == title].index.values[0]    


In [19]:
book_id = get_index("Harry Potter and the Half-Blood Prince (Book 6)")
book_id

2723

In [20]:
similar_books = list(enumerate(similarity[book_id]))
sorted_books = sorted(similar_books,key=lambda x:x[1],reverse=True)

In [21]:
for i in range(20):
  print(df[df.index == sorted_books[i][0]]["title"].values[0])   


Harry Potter and the Half-Blood Prince (Book 6)
Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Sorcerer's Stone (Book 1)
Harry Potter and the Order of the Phoenix (Book 5)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Prisoner of Azkaban
The Harry Potter Collection
Harry Potter and the Chamber of Secrets
From Far Away
Harry Potter and the Goblet of Fire
The Curse of the Mistwraith
In the Beginning
Angel Fire East
Star Wars: Clone Wars Volume 1 - The Defense of Kamino
First Meetings
Debt of Honor
Identity
A Hunger Like No Other
Henry IV Part Two
The Glass House


In [22]:
pd.DataFrame(similarity).to_csv("./similarity_books.csv")

In [24]:
type(similarity)

numpy.ndarray

In [25]:
import pickle
with open('similarity.pkl', 'wb') as obj:
    # Dump the numpy array to the file
    pickle.dump(similarity, obj)